In [1]:
# Import necessary libraries
from langchain_community.chat_models.friendli import get_chat_request
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_community.llms import Ollama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr
import os
from dotenv import load_dotenv
import time
import logging
import json
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
from dotenv import load_dotenv
from sympy import false
import random

from rameshm.llmeng.llmchat.llm_chat import LlmChat
from rameshm.llmeng.utils.init_utils import set_environment_logger

In [3]:
os.environ["LLM_KEY_FILE"] = "c:\learning\llm\projects\llm_engineering\.env"
logger = set_environment_logger()

Log File: c:\temp\my_logs.txt


In [4]:
def get_model(model_nm: str):
    model_nm = model_nm.split(" ")[1].strip()
    print(f"Using Model: {model_nm}")
    if "gpt" in model_nm:
        return ChatOpenAI(model=model_nm, api_key=os.getenv("OPENAI_API_KEY"), temperature=0.7, timeout=30)
    elif "claude" in model_nm:
        return ChatAnthropic(model=model_nm, api_key=os.getenv("ANTHROPIC_API_KEY"), timeout=30,
                             temperature=0.7, max_tokens=1024,top_p=0.9, top_k=40)
    elif "llama" in model_nm or "gemma" in model_nm:
        # Ollama run on "http://localhost:11434"  # Default Ollama URL. If you type that URL you shoud see "Ollama Running" message
        return Ollama(model=model_nm, # api_key="ollama",base_url="http://localhost:11434",
                      temperature=0.7, top_p=0.9, top_k=40, num_predict=256, repeat_penalty=1.1)
    elif "gemini" in model_nm:
         return ChatGoogleGenerativeAI(model=model_nm, google_api_key=os.getenv("GOOGLE_API_KEY"), timeout=30)
    else:
        raise Exception("Model: {model_nm} is not supported")

In [5]:
def validate_inputs(message: str, history: List, model: str, system_message: str) -> tuple:
    """Validate inputs before processing"""
    if not message or not message.strip():
        logger.error("Please enter a message")
        return false, "Please enter a message"

    if not model:
        logger.error("Please enter a model")
        return False, "Please select a model"

    # Check if required API keys are available
    model_name = model.lower()
    #  Ollama runs locally, so no API key is needed
    if "gpt" in model_name and not os.getenv("OPENAI_API_KEY"):
        msg = "OpenAI API key not found in environment variables"
        logger.error(msg)
        return False, msg
    elif "claude" in model_name and not os.getenv("ANTHROPIC_API_KEY"):
        msg = "Anthropic API key not found in environment variables"
        logger.error(msg)
        return False, msg
    elif "gemini" in model_name and not os.getenv("GOOGLE_API_KEY"):
        msg = "Google API key not found in environment variables"
        logger.error(msg)
        return False, msg
    return True, ""

In [6]:
def predict(message: str, history: List, selected_model: str, system_message: str,
            current_chat_id: Optional[str], chat_list: List[Dict[str, LlmChat]]) -> Tuple[str, List, List, str, List]:
    """Enhanced predict function with chat management"""
    start_time = time.time()
    chat_list_modified = chat_list
    err_msg = None  # set to string if we encounter an error

    try:
        # Validate inputs
        is_valid, error_msg = validate_inputs(message, history, selected_model, system_message)
        if not is_valid:
            logger.error(f"Validation failed: {error_msg}")
            raise ValueError(error_msg)

        logger.info(f"Processing request - Model: {selected_model}, Message length: {len(message)}")

        # Ensure history is a list
        if not isinstance(history, list):
            history = history.value if hasattr(history, 'value') else []

        # Initialize model
        model = get_model(selected_model)
        logger.info(f"Model initialized: {type(model).__name__}")

        # Build langchain history
        langchain_history = []

        # Add system message if provided
        if system_message and system_message.strip():
            langchain_history.append(SystemMessage(content=system_message.strip()))
            logger.info(f"Added system message ({len(system_message)} chars)")

        # Add conversation history
        for i, msg in enumerate(history):
            try:
                if msg.get('role') == "user":
                    langchain_history.append(HumanMessage(content=msg['content']))
                elif msg.get('role') == "assistant":
                    langchain_history.append(AIMessage(content=msg['content']))
            except Exception as e:
                err_msg = f"Malformed message history at index {i}: {msg}"
                logger.error(err_msg)
                raise ValueError(err_msg) from e

        # Add current user message
        langchain_history.append(HumanMessage(content=message))

        logger.debug(f"Built conversation with {len(langchain_history)} messages")
        logger.debug(f"Making API call with model: {selected_model} with total messages: {len(langchain_history)}")

        # Make API call with timeout handling
        llm_response = model.invoke(langchain_history)
        elapsed = time.time() - start_time
        logger.debug(f"API call completed in {elapsed:.2f}s")

        # Handle response format differences
        if isinstance(llm_response, str):
            response_content = llm_response
        else:
            response_content = getattr(llm_response, 'content', str(llm_response))

        if not response_content:
            err_msg = "Received empty response from model: {selected_model} for message: {message}"
            logger.error(err_msg)
            raise ValueError(err_msg)
        else:
            logger.info(f"Received response: {response_content}\n")

        # Update conversation history
        updated_history = history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": response_content}
        ]

        # Save/update chat
        if not current_chat_id:
            current_chat_id = generate_chat_id()
            current_llm_chat = LlmChat(chat_id=current_chat_id, history=updated_history,
                                       model_nm=selected_model, system_message=system_message)
            chat_list_modified += [{"chat_id": current_chat_id, "llm_chat": current_llm_chat}]
        else:
            # Update current with updated chat
            current_llm_chat = next((chat['llm_chat'] for chat in chat_list if chat['chat_id'] == current_chat_id), None)
            current_llm_chat.update_chat_history(updated_history)

        logger.debug(f"💬 Response: {response_content[:200]}{'...' if len(response_content) > 200 else ''}")
        return "", updated_history, updated_history, current_chat_id, get_chat_list(chat_list_modified),
    except Exception as e:
        elapsed = time.time() - start_time
        logger.error(f"Unexpected error after {elapsed:.2f}s: {e}")
        error_response = [
            {"role": "user", "content": message},
            {"role": "assistant", "content": f"🔥 ERROR Exception: Unexpected error: {str(e)}"}
        ]
        updated_history = history + error_response
        return "", updated_history, updated_history, current_chat_id or "", get_chat_list(chat_list_modified),
